# Healthcare Python Streaming Application Demo

This Python notebook shows how to create an app with Python that ingests and analyzes streaming data from a feed, and then visualizes the data in a notebook. You submit this app to be built in the Streaming Analytics service in IBM Bluemix. 

To create the app, you use the <a href="http://ibmstreams.github.io/streamsx.documentation//docs/python/1.6/python-appapi-devguide/index.html" target="_blank" rel="noopener noreferrer">IBM Streams Python API</a>. The API is provided in the streamsx package, which is integrated in DSX. You also use the API to access data streams from views defined in the app once it's running on the Streaming Analytics service.

The app subscribes to the *ingest-physionet* topic (<a href="https://www.physionet.org/" target="_blank" rel="noopener noreferrer">Physionet database</a>) containing the patient data, performs analysis on the patient data to calculate vital data for all patients, and finally creates a view for displaying the results in this notebook.

Submitting the Python app from the notebook to the Streaming Analytics service allows data to be retrieved from the view server. Once the data has been retrieved, it can be analyzed, manipulated, or visualized like any other data accessed from a notebook. In the case of this demo, waveform graphs and numerical widgets display the healthcare data of the patient.

This notebook runs in a **Python 3.5** environment. You should be familiar with Python and <a href="http://datascience.ibm.com/docs/content/getting-started/quick-overview.html#notebooks" target="_blank" rel="noopener noreferrer">Jupyter Notebooks</a>. This notebook also assumes that you are familiar with medical terms.

**Tip:** If you leave the notebook or the output or prompts are not displayed after running a cell, click **Kernel -> Restart** on the menu bar to rerun the cell.

The following diagram outlines the architecture of this demo. This notebook provides a pre-compiled IBM Streams application to simulate patient data, and publish the data to a topic shown in the **Ingest** box. The patient data is submitted to the ingest-physionet topic so that the data feed can be consumed by the app you're going to build later in this notebook. When running the cells of this notebook, you are creating the information shown in the **Analytics** and **Visualization** boxes of the diagram.

<img src='https://github.com/IBMStreams/streamsx.health/blob/develop/samples/HealthcareJupyterDemo/images/architecture_diagram.jpg?raw=true' alt="Demo Architecture" title="Demo Architecture"></img>

The data shown in the **Ingest** box of the diagram is obtained and processed for you to monitor and analyze the information in the **Visualization** box. The waveform data is shown in the graphs and numerical data is displayed in the black boxes. The **Poincare plot** graph above the black boxes analyzes the interval of the heartbeat's R wave to the next R wave (RR). The R wave is an upward deflection that is easily identified in the ECG. The R peak is the maximum amplitude of the R wave. This data is aggregated and continuously updated using the Streaming Analytics service.  

In Python, you can create a view object to connect the Python app to the view server. The **Streams View Server** shown in the **Analytics** box is the view function in the code which defines how Python can receive data from the view server. 
Each blue box is an operator in a Streams app. The operator then analyzes the data (tuples), sends it to the view server and to the downstream app. As a result, the live stream can be pulled as the data is being analyzed. The up arrow in the diagram indicates notebook requests to connect to the view server to receive data from a particular view. The down arrow retrieves the data to present it in the notebook as rendered graphics or visualizations.


## Table of contents

This notebook is divided into the following parts:

[Part 1: Setup](#setup)<br>
[Part 2: Create a data feed](#createfeed)<br>
[Part 3: Build a streaming app](#buildapp)<br>
[Part 4: Visualization](#visualization)<br>
[Part 5: Summary and next steps](#summary)<br>

<a id="setup"></a>
## Part 1: Setup

Complete the following steps to set up your DSX environment:

[1.1 Install the healthcare demo software package ](#install)<br>
[1.2 Start a Streaming Analytics service](#startservice)<br>
[1.3 Set up access to the service](#setupservice)<br>


<a id="install"></a>
### 1.1 Install the healthcare tutorial package

Run the following cell to install the latest software that contains Python functions required for this healthcare tutorial.  

In [ ]:
!pip install --user --upgrade "git+https://github.com/IBMStreams/streamsx.health.git#egg=healthdemo&subdirectory=samples/HealthcareJupyterDemo/package"

<a id="startservice"></a>
### 1.2 Start a Streaming Analytics service

Open the Bluemix web portal and <a href="https://console.ng.bluemix.net/" target="_blank" rel="noopener noreferrer">log in or sign up for a free Bluemix account</a>.

Ensure that your Streaming Analytics service is running in Bluemix.

If you don’t have a service, you can create one as follows:  
1. Click **Catalog** or **Create Service**, browse for `Streaming Analytics` and then click on it. <br>
1. Follow the instructions on the Streaming Analytics catalog page, type the *Service name* to set up your service. 
<img src='https://github.com/orzade/streamsx-notebooks/blob/master/servicename.png?raw=true' alt="Type your service name and click on Create" title="Streaming Analytics catalog - Type your service name"></img> 
1. Click **Create** to open the Streaming Analytics service dashboard. Your service starts automatically.

<a id="setupservice"></a>
### 1.3 Set up access to the service

You must provide the information that your streaming app needs to access the service.

Run the following cell, and enter your service credentials when prompted.

**Tip:** 
- To copy your service credentials, open the Streaming Analytics service dashboard click **Service Credentials**, then **View Credentials**, and finally click the Copy icon and paste your service credentials when prompted.<br>
<img src='https://github.com/orzade/streamsx-notebooks/blob/master/copyservicecredentials.png?raw=true' alt="Copy your service credentials" title="Streaming Analytics catalog - Copy your service credentials"></img>
<br>
- If you are not prompted to enter the service credentials, click **Kernel -> Restart** on the menu bar and rerun the cell.

In [ ]:
import json
import getpass
from streamsx.topology.context import ConfigParams
credentials=getpass.getpass('Streaming Analytics credentials:')

vs={'streaming-analytics': [{'name': 'Streaming Analytics', 'credentials': json.loads(credentials)}]}
config={ConfigParams.VCAP_SERVICES: vs, ConfigParams.SERVICE_NAME: 'Streaming Analytics'}

<a id="createfeed"></a>
## Part 2: Create a data feed

You can now submit the precompiled Physionet microservice to the Streaming Analytics service on IBM Bluemix to create a data feed:

1. Download the resources you need from <a href="https://github.com/IBMStreams/streamsx.health/releases" target="_blank" rel="noopener noreferrer">https://github.com/IBMStreams/streamsx.health/releases</a> to build the *PhysionetIngestServiceMulti.sab* application bundle file in your local system.
1. Open the **Streams Console** and click on **Submit Job** in the upper left part of the **Application Dashboard**.
<img src='https://github.com/orzade/streamsx-notebooks/blob/master/submitjob.png?raw=true' alt="Click Submit Job" title="Click Submit Job"></img>
1. On the **Submit Job** view select the application bundle file PhysionetIngestServiceMulti.sab from your local file system and click **Submit** to upload the file.
1. On the **Submission-Time Parameters** window set the number of patients to 3 or more in the **Value** field and click **OK**.


<a id="buildapp"></a>
## Part 3: Build a streaming app

Now you're ready to create the **HealthcareDemo** Python streaming application, and submit it to the Streaming Analytics service, where the app will be remotely built and deployed.

The following cell contains source code for the Python Topology application. This is a Python streaming application that ingests the patient data from the *ingest-physionet* topic (<a href="https://www.physionet.org/" target="_blank" rel="noopener noreferrer">Physionet database</a>) and performs analysis on the patient data to calculate vital data for all patients. It finally creates a view for displaying the result of the analysis.

In [ ]:
from streamsx.topology import schema
from streamsx.topology.topology import Topology
from streamsx.topology.context import submit

## The healthdemo package provides tools to analyze patient data
## See https://github.com/IBMStreams/streamsx.health/tree/develop/samples/HealthcareJupyterDemo/package
from healthdemo.patientmonitoring_functions import streaming_rpeak
from healthdemo.healthcare_functions import GenTimestamp, aggregate
from healthdemo.windows import SlidingWindow

topo = Topology('HealthcareDemo')

## The ingest-physionet provides data at a rate of 125 tuples/sec
sample_rate = 125

## Subscribe to the topic
patients_data = topo.subscribe('ingest-physionet', schema.CommonSchema.Json)
            
## Add timestamp to the data, so you can perform windowing
patients_data = patients_data.transform(GenTimestamp(sample_rate))

## Generate a window based on the provided sample_rate
patients_data = patients_data.transform(SlidingWindow(length=sample_rate, trigger=sample_rate-1))

## Aggregate the data within the window and create a tuple
patients_data = patients_data.transform(aggregate)

## Process data from 'ECG Lead II' and calculate RPeak and RR delta
patients_data = streaming_rpeak(patients_data, sample_rate, data_label='ECG Lead II')

## Create view for viewing patient vital data
patients_vital = patients_data.view(name='patients_vital')

rc = submit('STREAMING_ANALYTICS_SERVICE', topo, config)
print ("DONE")

<a id="visualization"></a>
## Part 4: Visualization

Complete the following steps to visualize the results of your app:

[4.1 Setup graphs for plotting patient vitals](#setupgraphs)<br>
[4.2 Provide data for the graphs](#providedata)<br>
[4.3 Display the graphs](#displaygraphs)<br>

<a id="setupgraphs"></a>
### 4.1 Setup graphs for plotting patient vitals

This cell initializes the nine graphs which will be used to display one patient's vital data.

Each property of the patient's vital data is identified by the signal label. Each graph is initialized by providing the signal label it plots and a title.

In [ ]:
from healthdemo.medgraphs import ECGGraph, PoincareGraph, NumericText, ABPNumericText

## Select which patient's data to plot
patientId = 'patient-1'

graph = {
    'leadII_poincare': PoincareGraph(signal_label='Poincare - ECG Lead II', title='Poincare - ECG Lead II'),
    'ecg_leadII_graph': ECGGraph(signal_label='ECG Lead II', title='ECG Lead II', 
                                 plot_width=600, min_range=-0.5, max_range=2.0),
    'ecg_leadV_graph': ECGGraph(signal_label='ECG Lead V', title='ECG Lead V', plot_width=600),
    'resp_graph': ECGGraph(signal_label='Resp', title='Resp', min_range=-1, max_range=3, plot_width=600),
    'pleth_graph': ECGGraph(signal_label='Pleth', title='Pleth', min_range=0, max_range=5, plot_width=600),
    'hr_numeric': NumericText(signal_label='HR', title='HR', color='#7cc7ff'),
    'pulse_numeric': NumericText(signal_label='PULSE', title='PULSE', color='#e71d32'),
    'spo2_numeric': NumericText(signal_label='SpO2', title='SpO2', color='#8cd211'),
    'abp_numeric': ABPNumericText(abp_sys_label='ABP Systolic', abp_dia_label='ABP Diastolic', 
                                  title='ABP', color='#fdd600')            
}

There is no output data displayed here.

<a id="providedata"></a>
### 4.2 Provide data for the graphs

This cell is responsible for propagating the graph objects with data in the view.

The view data contains vital data for all patients, and is continuously retrieved from the Streaming Analytics service in a background job.  Each graph object receives data for a specified patient. The graph objects extract and store the data that is relevant for that particular graph.

In [ ]:
from healthdemo.utils import get_patient_id

## load BokehJS visualization library (must be loaded in a separate cell)
from bokeh.io import output_notebook, push_notebook
from bokeh.resources import INLINE
output_notebook(resources=INLINE)
%autosave 0
%reload_ext autoreload
%autoreload 1

continue_data_collection = True

## retrieve data from Streams view in a background job
def data_collector(view, g):
    queue = view.start_data_fetch()
    while continue_data_collection:
        tup = queue.get()
        if patientId == get_patient_id(tup):
            for graphtype in g:
                g[graphtype].add(tup)
    view.stop_data_fetch()
            
from IPython.lib import backgroundjobs as bg
jobs = bg.BackgroundJobManager()
jobs.new(data_collector, patients_vital, graph)

<a id="displaygraphs"></a>
### 4.3 Display the graphs

This cell is responsible for laying out and displaying the graphs. 

Each time a call to ```update()``` is made on a graph object, the next data point is retrieved and displayed. Each graph object maintains an internal queue so that each time a call to ```update()``` is made, the next element in the queue is retrieved and removed.

There is a loop that continuously calls the ```update()``` method on each of the graphs for 60 seconds. After each graph has been updated, a call to ```push_notebook()``` is made, which causes the notebook to update the graphics.

In [ ]:
import time
from bokeh.io import show
from bokeh.layouts import column, row, widgetbox

## display graphs for a patient
show(
    row(
        column(
            graph['ecg_leadII_graph'].get_figure(), 
            graph['ecg_leadV_graph'].get_figure(), 
            graph['resp_graph'].get_figure(),
            graph['pleth_graph'].get_figure()
        ), 
        column(
            graph['leadII_poincare'].get_figure(),
            widgetbox(graph['hr_numeric'].get_figure()),
            widgetbox(graph['pulse_numeric'].get_figure()),
            widgetbox(graph['spo2_numeric'].get_figure()),
            widgetbox(graph['abp_numeric'].get_figure())
        )
    ),
    
    # If using bokeh > 0.12.2, uncomment the following statement
    #notebook_handle=True
)

## Timeout(in seconds) before stopping the graph
timeout = 60
endtime = time.time() + timeout

cnt = 0
while time.time() < endtime:
    ## update graphs
    for graphtype in graph:
        graph[graphtype].update()

    ## update notebook 
    cnt += 1
    if cnt % 5 == 0:
        push_notebook() ## refresh the graphs
        cnt = 0
    time.sleep(0.008)
    
# Stop data collection running in background thread
continue_data_collection = False

To plot the graph for a different patient, change `patientId` in [4.1 Setup graphs for plotting patient vitals](#setupgraphs), and rerun the cells.

<a id="summary"></a>
## Part 5: Summary and next steps

You learned how to create an app with Python that ingests and analyzes streaming data from a feed, and then visualizes the data in the notebook. You submitted this app to be built in the Streaming Analytics service in Bluemix.

Check out other notebooks in this series:

 - <a href="https://apsportal.ibm.com/exchange/public/entry/view/9fc33ce7301f10e21a9f92039ca9c6e8" target="_blank" rel="noopener noreferrer">Build a Python app on the Streaming Analytics service</a> 
 - <a href="https://apsportal.ibm.com/exchange/public/entry/view/9fc33ce7301f10e21a9f92039ca60bb7" target="_blank" rel="noopener noreferrer">Build and use a data model in real time with the Python API</a> 

Dig deeper:

* <a href="http://ibmstreams.github.io/streamsx.documentation/docs/latest/python/python-appapi-devguide/#developing-ibm-streams-applications-with-python" target="_blank" rel="noopener noreferrer">Developing IBM Streams applications with Python</a> 
* <a href="https://www.ibm.com/support/knowledgecenter/SSCRJU/SSCRJU_welcome.html" target="_blank" rel="noopener noreferrer">IBM Streams documentation</a> 
* <a href="https://github.com/IBMStreams/streamsx.health/tree/develop/samples/HealthcareJupyterDemo/package" target="_blank" rel="noopener noreferrer">IBM streamsx.health healthdemo</a> 


### Authors

James Cancilla is a software developer who specializes in streaming technology and cloud solutions.<br/>
Kendrick Wong is a software developer who specializes in streaming technology and cloud solutions.


Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.